<a href="https://colab.research.google.com/github/arekciminski/model_SW_SNN/blob/Dane_do_duzej_sieci_v2/SW_Logistic_KNN_DT_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ładowanie bibliotek 

In [ ]:
import os.path
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import RocCurveDisplay, roc_auc_score, roc_curve, confusion_matrix, accuracy_score, recall_score, f1_score, precision_score,balanced_accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier


#Ładowanie pliku z danymi z GitHub'a

In [ ]:
#file_name='dane_do_uczenia_1_1000000.zip'
'''if os.path.exists(file_name)==0:
  from google.colab import drive
  drive.mount("/content/drive/dane_do_uczenia_1_1000000.zip")'''
#!unzip "dane_do_uczenia_1_1000000.zip"


'if os.path.exists(file_name)==0:\n  from google.colab import drive\n  drive.mount("/content/drive/dane_do_uczenia_1_1000000.zip")'

In [ ]:
file_name="dane_do_uczenia_1_50000.csv"
#https://raw.githubusercontent.com/arekciminski/model_SW_SNN/Dane_do_duzej_sieci_v2/dane_do_uczenia_1_50000.csv
if os.path.exists(file_name)==0:
  #!wget "https://raw.githubusercontent.com/arekciminski/model_SW_SNN/Dane_do_duzej_sieci_v2/dane_do_uczenia_4000000.csv"
  !wget 'https://raw.githubusercontent.com/arekciminski/model_SW_SNN/Dane_do_duzej_sieci_v2/dane_do_uczenia_1_50000.csv'

#Ładowanie pliku lokalnego do pamięci


In [ ]:
df= pd.read_csv(file_name,decimal =',',sep=';')

In [ ]:
df.shape

(50000, 19)

In [ ]:
df.head()

,Unnamed: 0,error_output,time_output,head_output_083,head_output_158,head_output_180,flow_output_F1,flow_output_K1,flow_output_P1,tank_output_180,energy_output_F1,energy_output_K1,energy_output_P1,tank_input_180,pump_input_F1,pump_input_K1,pump_input_P1,demand_input_par2,demand_input_kar2
0,0,0,0,199.933365,199.647156,167.500000,138.377930,83.717445,2.380052,167.500000,20.431585,10.736735,0.539035,1.5,0.71,0.73,0.71,0.456,0.362
1,1,4,0,220.021545,219.745071,168.699997,177.392029,112.017647,0.000000,168.699997,28.465670,23.276525,0.000000,2.7,0.77,0.92,0.85,0.377,0.524
2,2,0,0,194.770889,194.528488,170.199997,205.863663,13.969801,90.367699,170.199997,35.464478,3.120016,6.956906,4.2,0.83,0.63,0.77,0.586,0.476
3,3,4,0,218.792007,218.509995,170.500000,0.000000,99.217644,0.000000,170.500000,0.000000,18.729397,0.000000,4.5,0.52,0.89,0.64,0.352,0.444
4,4,0,0,190.849152,190.601852,168.899994,264.401123,63.869576,62.547928,168.899994,47.016254,6.140797,4.026922,2.9,0.95,0.60,0.63,0.562,0.614


#Losowa zmiana pozycji wierszy w df


In [ ]:
df = df.sample(frac = 1)

In [ ]:
df['error_output'].value_counts()

4    40068
0     9932
Name: error_output, dtype: int64

#Podział danych na X i y

In [ ]:
df.columns

Index(['Unnamed: 0', 'error_output', 'time_output', 'head_output_083',
       'head_output_158', 'head_output_180', 'flow_output_F1',
       'flow_output_K1', 'flow_output_P1', 'tank_output_180',
       'energy_output_F1', 'energy_output_K1', 'energy_output_P1',
       'tank_input_180', 'pump_input_F1', 'pump_input_K1', 'pump_input_P1',
       'demand_input_par2', 'demand_input_kar2'],
      dtype='object')

In [ ]:
x_columns =['tank_input_180', 'pump_input_F1', 'pump_input_K1', 'pump_input_P1',
       'demand_input_par2', 'demand_input_kar2']
y_column = ['error_output']

In [ ]:
X = df[x_columns]
y = df[y_column].replace([4,6],[1,1])
y

,error_output
37208,0
20146,1
37840,1
49522,1
19399,1
...,...
38477,1
4294,1
6826,1
885,1


#Podział danych na testowe i treningowe

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#Model regresji logistycznej

In [ ]:
model_LR = LogisticRegression( max_iter = 5000)
model_LR.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=5000)

#Predykcja na podstawie modelu

In [ ]:
y_pred_LR = model_LR.predict(X_test)
y_scores_LR = model_LR.predict_proba(X_test)

#Macierz pomyłek

In [ ]:
cm = confusion_matrix(y_test, y_pred_LR)
cm
#sns.heatmap(cm,annot = True)

array([[  656,  2339],
       [  612, 11393]])

#Wskaźniki

In [ ]:
acc_test_LR = accuracy_score(y_test, y_pred_LR)
b_acc_test_LR = balanced_accuracy_score(y_test, y_pred_LR)
prec_test_LR = precision_score(y_test, y_pred_LR)
f1_test_LR = f1_score(y_test, y_pred_LR)

print(f"Accuracy:{acc_test_LR:0.4f}" )
print(f"Balance Accuracy:{b_acc_test_LR:0.4f}" )
print(f"Preccision:{prec_test_LR:0.4f}" )
print(f"F1 test:{f1_test_LR:0.4f}" )

Accuracy:0.8033
Balance Accuracy:0.5840
Preccision:0.8297
F1 test:0.8853


In [ ]:
from joblib import dump, load
dump(model_LR, 'LR_best_model.joblib')

['LR_best_model.joblib']

In [ ]:
scaler = MinMaxScaler()

# "Nauczenie" (sfitowanie) skalera
scaler.fit(X)

X_train_scaled_KNN = scaler.transform(X_train)
X_test_scaled_KNN = scaler.transform(X_test)

# FYI
X_train_scaled_with_names = pd.DataFrame(X_train_scaled_KNN, index = X_train.index, columns = X_train.columns)
X_test_scaled_with_names = pd.DataFrame(X_test_scaled_KNN, index = X_test.index, columns = X_train.columns)

In [ ]:
param_grid_knn ={
    
    'n_neighbors' : list(range(3,8)),
    'weights' : ['uniform', 'distance'],
    'algorithm' : ['ball_tree', 'kd_tree', 'brute'],
    'metric' : ['euclidean']
}

In [ ]:
#knn = KNeighborsClassifier()

#GS = GridSearchCV(knn, param_grid_knn,verbose=1)

#GS.fit(X_train_scaled_KNN, y_train.values.ravel())



In [ ]:
#print(GS.best_params_)

In [ ]:
best_params_={'algorithm': 'ball_tree', 'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'distance'}

In [ ]:
#knn_best  = KNeighborsClassifier(algorithm = GS.best_params_['algorithm'], metric = GS.best_params_['metric'],
#                           n_neighbors = GS.best_params_['n_neighbors'], weights = GS.best_params_['weights'])

knn_best  = KNeighborsClassifier(algorithm = best_params_['algorithm'], metric = best_params_['metric'],
                           n_neighbors = best_params_['n_neighbors'], weights = best_params_['weights'])

In [ ]:
knn_best.fit(X_train_scaled_KNN, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(algorithm='ball_tree', metric='euclidean', n_neighbors=7,
                     weights='distance')

In [ ]:
y_pred_KNN  = knn_best.predict(X_test_scaled_KNN)

In [ ]:
cm = confusion_matrix(y_test, y_pred_KNN)
cm

array([[ 2543,   452],
       [  346, 11659]])

In [ ]:
acc_test_KNN = accuracy_score(y_test, y_pred_KNN)
b_acc_test_KNN = balanced_accuracy_score(y_test, y_pred_KNN)
prec_test_KNN = precision_score(y_test, y_pred_KNN)
f1_test_KNN = f1_score(y_test, y_pred_KNN)

print(f"Accuracy:{acc_test_KNN:0.4f}" )
print(f"Balance Accuracy:{b_acc_test_KNN:0.4f}" )
print(f"Preccision:{prec_test_KNN:0.4f}" )
print(f"F1 test:{f1_test_KNN:0.4f}" )

Accuracy:0.9468
Balance Accuracy:0.9101
Preccision:0.9627
F1 test:0.9669


In [ ]:
from joblib import dump, load
dump(knn_best, 'KNN_best_model.joblib') 

['KNN_best_model.joblib']

In [ ]:
param_grid_dt = {
    'criterion' :['gini', 'entropy', 'log_loss'],
    'max_depth' : list(range(2,20,4)),
    'min_samples_split': list(range(2,50, 5))
}

In [ ]:
'''dt = DecisionTreeClassifier()


GS = GridSearchCV(dt, param_grid_dt)

GS.fit(X_train, y_train)'''


'dt = DecisionTreeClassifier()\n\n\nGS = GridSearchCV(dt, param_grid_dt)\n\nGS.fit(X_train, y_train)'

In [ ]:
#GS.best_params_

In [ ]:
best_params_ = {'criterion': 'gini', 'max_depth': 18, 'min_samples_split': 2}

In [ ]:
#dt_best = DecisionTreeClassifier(criterion=GS.best_params_['criterion'],max_depth = GS.best_params_['max_depth'])
dt_best = DecisionTreeClassifier(criterion=best_params_['criterion'],max_depth = best_params_['max_depth'])

In [ ]:
dt_best.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=18)

In [ ]:
y_pred_test_dt = dt_best.predict(X_test)
y_pred_train_dt = dt_best.predict(X_train)

In [ ]:
acc_test_dt = accuracy_score(y_test, y_pred_test_dt)
b_acc_test_dt = balanced_accuracy_score(y_test, y_pred_test_dt)
prec_test_dt = precision_score(y_test, y_pred_test_dt)
f1_test_dt = f1_score(y_test, y_pred_test_dt)

print(f"Accuracy:{acc_test_dt:0.4f}" )
print(f"Balance Accuracy:{b_acc_test_dt:0.4f}" )
print(f"Preccision:{prec_test_dt:0.4f}" )
print(f"F1 test:{f1_test_dt:0.4f}" )

Accuracy:0.9672
Balance Accuracy:0.9501
Preccision:0.9804
F1 test:0.9795


In [ ]:
from joblib import dump, load
dump(dt_best, 'DT_best_model.joblib')

['DT_best_model.joblib']